# <font color = '#00CCFF'> High Value Customer Identification </font>

Project Objective: Find significant customers for the business who make high purchases of their favourite products. The organization wants to roll out a loyalty program to the high-value customers after identification of segments. Use the clustering methodology to segment customers into groups:

Dataset Description: This is a transnational dataset that contains all the transactions occurring between Nov-2016 to Dec-2017 for a UK-based online retail store.

Attribute Description:
- InvoiceNo: Invoice number (A 6-digit integral number uniquely assigned to each transaction)
- StockCode: Product (item) code
- Description: Product (item) name
- Quantity: The quantities of each product (item) per transaction
- InvoiceDate: The day when each transaction was generated
- UnitPrice: Unit price (Product price per unit)
- CustomerID: Customer number (Unique ID assigned to each customer)
- Country: Country name (The name of the country where each customer resides)

# 0. Imports

## 0.1. Libraries

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import scikitplot as skplt
import plotly.express as px
#%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

import umap.umap_ as umap

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

from ydata_profiling import ProfileReport

## 0.2. Functions

In [2]:
# Save intermediate dataset
def save_interim(dataframe, file_name):
   dataframe.to_csv('../data/interim/'+file_name+'.zip', index=False, compression= 'zip')

# Load intermediate dataset
def load_interim(file_name):
   return pd.read_csv('../data/interim/'+file_name, compression='zip')

## 0.3. Load dataset

In [3]:
path = '/home/ezequiel/Documentos/Comunidade_DS/insiders_clustering/data/raw/Ecommerce.zip'
df_raw = pd.read_csv(path, encoding='unicode_escape', compression='zip')
df_raw = df_raw.drop(columns=['Unnamed: 8'], axis=1)

In [4]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom


# 1. Descrição dos dados

In [5]:
df1 = df_raw.copy()

## 1.1. Rename columns

In [6]:
df1.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [7]:
cols = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date', 'unit_price', 'customer_id', 'country']
df1.columns = cols

## 1.2. Data dimensions

In [8]:
print(f'Number of rows: {df1.shape[0]}')
print(f'Number of columns: {df1.shape[1]}')

Number of rows: 541909
Number of columns: 8


## 1.3. Data types

InvoiceNo should be a number, but some records have letters

In [9]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4. Check NA

In [10]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5. Replace NA

In [11]:
# Procurar se há algum customer_id "perdido" a partir do invoice_no
df_missing = df1[df1['customer_id'].isna()]
df_not_missing = df1[~df1['customer_id'].isna()]

In [12]:
missing_invoice = df_missing['invoice_no'].drop_duplicates().tolist()

In [13]:
# Os invoice_no dos customer_id faltantes não correspondem a nenhum invoice_no que possui um customer_id
df_not_missing.loc[df_not_missing['invoice_no'].isin(missing_invoice), :]

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country


In [14]:
df_not_missing['customer_id'].max()

18287.0

In [15]:
# Alternativa: atribuir um customer_id não utilizado para cada invoice_no único que não possui customer_id
aux = df_missing[['invoice_no']].drop_duplicates()
aux['customer_id'] = np.arange(19000, 19000+len(aux), 1)

df1 = pd.merge(df1, aux, on='invoice_no', how='left')
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])
df1.drop(columns=['customer_id_x', 'customer_id_y'], inplace=True)

In [16]:
df1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

In [17]:
df1 = df1.dropna(subset=['description', 'customer_id'])

## 1.6. Change dtyper

In [18]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
country          object
customer_id     float64
dtype: object

In [19]:
# invoice_date -> to datetime
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

# customer_id -> to int
df1['customer_id'] = df1['customer_id'].astype(int)

In [20]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

In [21]:
save_interim(dataframe=df1, file_name='df1.csv')

## 1.7. Descriptive statistics

In [22]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(include='object')

### 1.7.1. Numerical attributes

In [23]:
df_describe = num_attributes.describe().T
df_describe['range'] = df_describe['max'] - df_describe['min']
df_describe['median'] = num_attributes.median()
df_describe['skewness'] = num_attributes.skew()
df_describe['kurtosis'] = num_attributes.kurtosis()
df_describe.reset_index()

,index,count,mean,std,min,25%,50%,75%,max,range,median,skewness,kurtosis
0,quantity,540455.0,9.603129,218.007598,-80995.00,1.00,3.00,10.00,80995.0,161990.00,3.00,-0.281253,120252.449063
1,unit_price,540455.0,4.623519,96.889628,-11062.06,1.25,2.08,4.13,38970.0,50032.06,2.08,186.257919,58848.017575
2,customer_id,540455.0,16678.153600,2907.572952,12346.00,14364.00,16241.00,18257.00,22709.0,10363.00,16241.00,0.493600,-0.792496


<font color='red'>**OBSERVAÇÕES:**</font>
- quantity com mínimo negativo (devolução?)
- quantity com valor max e min iguais em módulo
- unit_price com min = zero (promoção?)

### 1.7.2. Categorical attributes

#### Invoice_no

In [24]:
# verifica se invoice_no contém, ao menos, uma letra e agrupa por invoice_no para o dataframe inteiro
aux = df1[df1['invoice_no'].str.contains('[a-zA-Z]')].groupby('invoice_no').count().reset_index()
print(f'Qtd de invoices únicos com letras: {len(aux)}')
print(f'Qtd total de invoices com letras {aux["stock_code"].sum()}')

Qtd de invoices únicos com letras: 3839
Qtd total de invoices com letras 9291


In [25]:
aux = len(df1.loc[df1['invoice_no'].str.contains('[a-zA-Z]'), ['quantity']])
print(f'Qtd de invoices com letras onde a quantidade é negativa: {aux}')

Qtd de invoices com letras onde a quantidade é negativa: 9291


#### Stock_code

In [26]:
# verifica se stock_code contém, ao menos, uma letra e conta quantos são
df1[df1['stock_code'].str.contains('[a-zA-Z]')]

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2016-11-29,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2016-11-29,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2016-11-29,3.39,United Kingdom,17850
45,536370,POST,POSTAGE,3,2016-11-29,18.00,France,12583
...,...,...,...,...,...,...,...,...
541778,581578,84997C,CHILDRENS CUTLERY POLKADOT BLUE,8,2017-12-07,4.15,Germany,12713
541809,581579,85099C,JUMBO BAG BAROQUE BLACK WHITE,10,2017-12-07,1.79,United Kingdom,17581
541838,581580,84993A,75 GREEN PETIT FOUR CASES,2,2017-12-07,0.42,United Kingdom,12748
541844,581580,85049A,TRADITIONAL CHRISTMAS RIBBONS,1,2017-12-07,1.25,United Kingdom,12748


In [27]:
# verifica se stock_code contém somente letras e mostra a quantidade de ocorrências
df1.loc[~df1['stock_code'].str.contains('[0-9]'), 'stock_code'].value_counts()

stock_code
POST            1252
DOT              709
M                571
D                 77
S                 63
BANK CHARGES      37
AMAZONFEE         34
CRUK              16
DCGSSGIRL         13
DCGSSBOY          11
PADS               4
B                  3
m                  1
Name: count, dtype: int64

In [28]:
df1.loc[~df1['stock_code'].str.contains('[0-9]'), 'stock_code'].unique()

array(['POST', 'D', 'DOT', 'M', 'BANK CHARGES', 'S', 'AMAZONFEE', 'm',
       'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

#### Country

In [29]:
df1['country'].value_counts(normalize=True)

country
United Kingdom          0.914089
Germany                 0.017569
France                  0.015833
EIRE                    0.015165
Spain                   0.004687
Netherlands             0.004387
Belgium                 0.003828
Switzerland             0.003704
Portugal                0.002811
Australia               0.002330
Norway                  0.002009
Italy                   0.001486
Channel Islands         0.001403
Finland                 0.001286
Cyprus                  0.001151
Sweden                  0.000855
Unspecified             0.000825
Austria                 0.000742
Denmark                 0.000720
Japan                   0.000662
Poland                  0.000631
Israel                  0.000550
USA                     0.000538
Hong Kong               0.000533
Singapore               0.000424
Iceland                 0.000337
Canada                  0.000279
Greece                  0.000270
Malta                   0.000235
United Arab Emirates    0.000126
Eu

<font color='red'>**OBSERVAÇÕES:**</font>
- invoice_no possui:
    - 3839 entradas únicas que possuem letras
    - 9291 entradas totais que possuem letras
    - todas as entradas com letras correspondem à uma quantity negativa, indicando uma operação de cancelamento de venda
- stock_code que possuem somente letras:
    - BANK CHARGES -> qtd: 37
    - AMAZONFEE ->    qtd: 34
    - CRUK ->         qtd: -16
    - D ->            qtd: 77
    - DOT ->          qtd: 710
    - M ->            qtd: 571
    - S ->            qtd: 63
    - POST ->         qtd: 1256
    - DCGSSGIRL ->    qtd: 13
    - DCGSSBOY ->     qtd: 11
    - PADS ->         qtd: 4
    - B ->            qtd: 3
    - m ->            qtd: 1
- description:
    - deletar coluna pois, a princípio, não tem relevância como feature
- country:
    - 91,43% está no Reino Unido

In [ ]:
#profile = ProfileReport(df=df1, title='Profiling report')

In [ ]:
#profile

# 2. Variables Filtering

In [30]:
df2 = df1.copy()

In [31]:
df2.query('quantity < 0').sort_values('quantity').head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2017-12-07,2.08,United Kingdom,16446
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2017-01-16,1.04,United Kingdom,12346
225529,556690,23005,printing smudges/thrown away,-9600,2017-06-12,0.00,United Kingdom,20914
225530,556691,23005,printing smudges/thrown away,-9600,2017-06-12,0.00,United Kingdom,20915
4287,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2016-11-30,0.03,United Kingdom,15838


In [32]:
df2.query('customer_id == 20914')

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
225529,556690,23005,printing smudges/thrown away,-9600,2017-06-12,0.0,United Kingdom,20914


In [33]:
# selecionar somente preços maiores que 4 centavo
df2 = df2.query('unit_price >= 0.04')

In [34]:
# retirar alguns stock_codes
codes = ['POST', 'D', 'DOT', 'M', 'BANK CHARGES', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK']
df2 = df2.query('stock_code != @codes')

In [35]:
df2 = df2.drop(columns='description')

In [36]:
drop_country = ['European Community', 'Unspecified']
df2 = df2.query('country != @drop_country')

In [37]:
# separar as quantidades negativas como devoluções (cancelamentos)
df2_returns = df2.query('quantity < 0')
df2_purchases = df2.query('quantity >= 0')

# 3. Feature Engineering

In [108]:
df3 = df2.copy()

In [109]:
# data reference - group by customer
df_ref = df3[['customer_id']].drop_duplicates(ignore_index=True)
df_ref.head()

,customer_id
0,17850
1,13047
2,12583
3,13748
4,15100


#### **Gross revenue (Faturamento)**

Faturamento para cada produto = quantidade x preço

In [110]:
df2_purchases['gross_revenue'] = df2_purchases['quantity'] * df2_purchases['unit_price']

/tmp/ipykernel_25214/2099251183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_purchases['gross_revenue'] = df2_purchases['quantity'] * df2_purchases['unit_price']


In [111]:
# total value of purchases per customer
df_monetary = df2_purchases[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

#### **Recency**

How recent is the last purchase made by each customer.

In [112]:
# find the last purchase date of each customer and subtracts of the overall last purchase date
df_last_purchase = df2_purchases[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_last_purchase['recency'] = (df_last_purchase['invoice_date'].max() - df_last_purchase['invoice_date']).dt.days
df_ref = pd.merge(df_ref, df_last_purchase, on='customer_id', how='left')

In [113]:
df_ref.isna().sum()

customer_id       0
gross_revenue    67
invoice_date     67
recency          67
dtype: int64

#### **Quantity of purchases**

Consider the unique invoice_no (transaction)

In [114]:
df_freq = df2_purchases.loc[:, ['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index().rename(columns={'invoice_no': 'qtd_invoices'})
df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')
df_ref.head()

,customer_id,gross_revenue,invoice_date,recency,qtd_invoices
0,17850,5391.21,2016-11-30,372.0,34.0
1,13047,3232.59,2017-10-12,56.0,9.0
2,12583,6705.38,2017-12-05,2.0,15.0
3,13748,948.25,2017-09-03,95.0,5.0
4,15100,876.00,2017-01-08,333.0,3.0


In [115]:
df_ref.isna().sum()

customer_id       0
gross_revenue    67
invoice_date     67
recency          67
qtd_invoices     67
dtype: int64

#### **Quantity of items purchased**

In [116]:
aux = df2_purchases.loc[:, ['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'qtd_items'})
df_ref = pd.merge(df_ref, aux, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    67
invoice_date     67
recency          67
qtd_invoices     67
qtd_items        67
dtype: int64

#### **Quantity of products purchased**

In [117]:
aux = df2_purchases.loc[:, ['customer_id', 'stock_code']].groupby('customer_id').count().reset_index().rename(columns={'stock_code': 'qtd_products'})
df_ref = pd.merge(df_ref, aux, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    67
invoice_date     67
recency          67
qtd_invoices     67
qtd_items        67
qtd_products     67
dtype: int64

#### **Average ticket**

Average spend of each client

In [118]:
df_avg_ticket = df2_purchases[['customer_id', 'gross_revenue']].groupby('customer_id').mean().reset_index().rename(columns={'gross_revenue': 'avg_ticket'})
df_ref = pd.merge(df_ref, df_avg_ticket, how='left', on='customer_id')

In [119]:
df_ref.isna().sum()

customer_id       0
gross_revenue    67
invoice_date     67
recency          67
qtd_invoices     67
qtd_items        67
qtd_products     67
avg_ticket       67
dtype: int64

#### **Average recency days**

Average days between purchases

In [120]:
df_aux = df3[['customer_id', 'invoice_date']].drop_duplicates().sort_values(['customer_id', 'invoice_date'], ascending=[False, False])
df_aux['next_customer_id'] = df_aux['customer_id'].shift()
df_aux['previous_date'] = df_aux['invoice_date'].shift()

df_aux['avg_recency_days'] = df_aux.apply(lambda x: (x['invoice_date'] - x['previous_date']).days*-1 if x['customer_id'] == x['next_customer_id'] else np.nan, axis=1)

df_aux = df_aux.drop(['invoice_date', 'next_customer_id', 'previous_date'], axis=1).dropna()

df_avg_recency_days = df_aux.groupby('customer_id').mean().reset_index()

df_ref = pd.merge(df_ref, df_avg_recency_days, on='customer_id', how='left')
df_ref.isna().sum()

customer_id            0
gross_revenue         67
invoice_date          67
recency               67
qtd_invoices          67
qtd_items             67
qtd_products          67
avg_ticket            67
avg_recency_days    2794
dtype: int64

#### **Frequency purchase**

In [121]:
df_aux = df3[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates().groupby('customer_id').agg(
                                                            max_inv_date = ('invoice_date', max),
                                                            min_inv_date = ('invoice_date', min),
                                                            days = ('invoice_date', lambda x: (x.max() - x.min()).days + 1),
                                                            buy = ('invoice_no', 'count')).reset_index()

In [122]:
# Frequency
df_aux['frequency'] = df_aux[['buy', 'days']].apply(lambda x: x['buy']/x['days'] if x['days'] != 0 else 0, axis=1)

In [123]:
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left')

df_ref.isna().sum()

customer_id            0
gross_revenue         67
invoice_date          67
recency               67
qtd_invoices          67
qtd_items             67
qtd_products          67
avg_ticket            67
avg_recency_days    2794
frequency              0
dtype: int64

#### **Number of returns**

In [124]:
df_returns = df2_returns[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'returns'})
df_returns['returns'] = df_returns['returns'] * -1

In [125]:
df_ref= pd.merge(df_ref, df_returns, on='customer_id', how='left')
df_ref.loc[df_ref['returns'].isna(), 'returns'] = 0

In [126]:
df_ref.isna().sum()

customer_id            0
gross_revenue         67
invoice_date          67
recency               67
qtd_invoices          67
qtd_items             67
qtd_products          67
avg_ticket            67
avg_recency_days    2794
frequency              0
returns                0
dtype: int64

#### **Basket size** (quantidade de itens por cesta)

In [127]:
df_aux = df2_purchases.loc[:, ['customer_id', 'invoice_no', 'quantity']].groupby('customer_id').agg(
                                                                        n_purchase = ('invoice_no', 'nunique'),
                                                                        n_products = ('quantity', 'sum')).reset_index()

In [128]:
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

In [129]:
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'avg_basket_size']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id            0
gross_revenue         67
invoice_date          67
recency               67
qtd_invoices          67
qtd_items             67
qtd_products          67
avg_ticket            67
avg_recency_days    2794
frequency              0
returns                0
avg_basket_size       67
dtype: int64

#### **Unique Basket size** (quantidade de itens distintos por cesta)

In [130]:
df_aux = df2_purchases.loc[:, ['customer_id', 'invoice_no', 'stock_code']].groupby('customer_id').agg(
                                                                        n_purchase = ('invoice_no', 'nunique'),
                                                                        n_products = ('stock_code', 'count')).reset_index()

In [131]:
df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

In [132]:
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'avg_unique_basket_size']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id                  0
gross_revenue               67
invoice_date                67
recency                     67
qtd_invoices                67
qtd_items                   67
qtd_products                67
avg_ticket                  67
avg_recency_days          2794
frequency                    0
returns                      0
avg_basket_size             67
avg_unique_basket_size      67
dtype: int64

In [133]:
df_ref.drop(columns=['invoice_date'], inplace=True)
df_ref.head()

,customer_id,gross_revenue,recency,qtd_invoices,qtd_items,qtd_products,avg_ticket,avg_recency_days,frequency,returns,avg_basket_size,avg_unique_basket_size
0,17850,5391.21,372.0,34.0,1733.0,297.0,18.152222,35.500000,0.486111,40.0,50.970588,8.735294
1,13047,3232.59,56.0,9.0,1390.0,171.0,18.904035,27.250000,0.048780,35.0,154.444444,19.000000
2,12583,6705.38,2.0,15.0,5028.0,232.0,28.902500,23.187500,0.045699,50.0,335.200000,15.466667
3,13748,948.25,95.0,5.0,439.0,28.0,33.866071,92.666667,0.017921,0.0,87.800000,5.600000
4,15100,876.00,333.0,3.0,80.0,3.0,292.000000,8.600000,0.136364,22.0,26.666667,1.000000


# 4. EDA (Exploratory Data Analysis)

In [134]:
df4 = df_ref.dropna()
df4.isna().sum()

customer_id               0
gross_revenue             0
recency                   0
qtd_invoices              0
qtd_items                 0
qtd_products              0
avg_ticket                0
avg_recency_days          0
frequency                 0
returns                   0
avg_basket_size           0
avg_unique_basket_size    0
dtype: int64

## 4.1. Univariate Analysis

In [135]:
profile = ProfileReport(df4)
profile.to_file('../reports/output.html')

/home/ezequiel/.pyenv/versions/ins_clustering/lib/python3.10/site-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### 4.1.1. Gross revenue

In [136]:
df4.sort_values('gross_revenue', ascending=False).head()

,customer_id,gross_revenue,recency,qtd_invoices,qtd_items,qtd_products,avg_ticket,avg_recency_days,frequency,returns,avg_basket_size,avg_unique_basket_size
1014,14646,279138.02,1.0,72.0,196844.0,2060.0,135.503893,7.844444,0.206215,288.0,2733.944444,28.611111
490,18102,259657.30,0.0,60.0,64124.0,431.0,602.453132,14.680000,0.163043,0.0,1068.733333,7.183333
432,17450,194550.79,8.0,46.0,69993.0,337.0,577.302047,12.821429,0.136111,952.0,1521.586957,7.326087
3325,16446,168472.50,0.0,2.0,80997.0,3.0,56157.500000,205.000000,0.014563,80995.0,40498.500000,1.500000
66,14911,140450.72,1.0,199.0,80263.0,5673.0,24.757751,2.601399,0.651475,3332.0,403.331658,28.507538


In [96]:
df1.query('customer_id == 14646')

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
37952,539491,21981,PACK OF 12 WOODLAND TISSUES,12,2016-12-18,0.29,Netherlands,14646
37953,539491,21986,PACK OF 12 PINK POLKADOT TISSUES,12,2016-12-18,0.29,Netherlands,14646
37954,539491,22720,SET OF 3 CAKE TINS PANTRY DESIGN,2,2016-12-18,4.95,Netherlands,14646
37955,539491,21931,JUMBO STORAGE BAG SUKI,1,2016-12-18,1.95,Netherlands,14646
37956,539491,22613,PACK OF 20 SPACEBOY NAPKINS,2,2016-12-18,0.85,Netherlands,14646
...,...,...,...,...,...,...,...,...
534958,581176,22908,PACK OF 20 NAPKINS RED APPLES,96,2017-12-05,0.72,Netherlands,14646
534959,581176,22907,PACK OF 20 NAPKINS PANTRY DESIGN,96,2017-12-05,0.72,Netherlands,14646
534960,581176,22029,SPACEBOY BIRTHDAY CARD,72,2017-12-05,0.36,Netherlands,14646
534961,581176,22712,CARD DOLLY GIRL,72,2017-12-05,0.36,Netherlands,14646


## 4.2. Bivariate Analysis

In [ ]:
df2.hist(bins=50, figsize=(10,6))

In [ ]:
df_ref.hist(bins=75, figsize=(10,6))

In [ ]:
df4 = df_ref.dropna().copy()
df4.isna().sum()

# 5. Data Preparation

In [ ]:
df5 = df4.copy()

In [ ]:
std_scaler =StandardScaler()

df5['gross_revenue'] = std_scaler.fit_transform(df5[['gross_revenue']])
df5['recency'] = std_scaler.fit_transform(df5[['recency']])
df5['invoice_no'] = std_scaler.fit_transform(df5[['invoice_no']])
df5['avg_ticket'] = std_scaler.fit_transform(df5[['avg_ticket']])
#df5['returns'] = std_scaler.fit_transform(df5[['returns']])

# 6. Feature Selection

In [ ]:
df6 = df5.copy()

# 7. Hyperparameter Fine Tunning

In [ ]:
df7 = df5.drop(columns=['returns'])
df7.head()

In [ ]:
df7.describe()

## 7.1. Within-Cluster Sum of Squares (WSS)

Returns the compactness of the clusters. It is global measure and does not tell if the clusters are well separated or if there is overlaping between them.

In [ ]:
clusters = [2, 3, 4, 5, 6, 7]
model = KMeans(n_init=10)
elbow = KElbowVisualizer(estimator=model, k=clusters)
elbow.fit(df7)
elbow.show()

In [ ]:
S = KElbowVisualizer(estimator=model, k=clusters, metric='silhouette', timings=False)
S.fit(df7)
S.show()

## 7.2. Silhouette Analysis

Measures how well a point fits into its cluster compared to the others.

In [ ]:
fig, ax = plt.subplots(3,2, figsize=(16,20))

for k in clusters:
    km_model = KMeans(n_clusters=k, init='random', n_init=10, max_iter=100, random_state=42)
    x, y = divmod(k, 2)
    SS = SilhouetteVisualizer(estimator=km_model, colors='yellowbrick', ax=ax[x-1][y])
    SS.fit(df7)
    SS.finalize()
    #SS.show()

# 8. Model Training

## 8.1. K-Means

In [ ]:
k = 3

kmeans = KMeans(n_clusters=k, init='random', n_init=10, max_iter=300, random_state=42)
kmeans.fit(df7)
labels = kmeans.predict(df7)

### 8.1.1. Validation

In [ ]:
# WSS
print(f'WSS: {kmeans.inertia_}')

# SS
print(f'Silhouette score: {silhouette_score(X=df7, labels=labels, metric="euclidean")}')

# 9. Cluster Analisys

In [ ]:
df9 = df7.copy()
df9['cluster'] = labels
df9.head()

## 9.1. Visualization

In [ ]:
df7.head()

In [ ]:
SS_fig = SilhouetteVisualizer(kmeans, colors='sns_dark')
SS_fig.fit(df7)
SS_fig.show()

In [ ]:
#fig = px.scatter_3d(data_frame=df9, x='gross_revenue', y='recency', z='invoice_no', color='cluster', width=600, height=600)
#fig.show()

In [ ]:
df9.head()

### 9.1.1. 2d plot

In [ ]:
df_viz = df9.drop(columns='customer_id')
sns.pairplot(data=df_viz, hue='cluster', corner=True, palette='Set1')

### 9.1.2. UMAP

Ferramenta para visualização de dados com alta dimensionalidade em um plano 2D.
- Clusters com boa definição (bom agrupamento) em 2D indicam um bom agrupamento na alta dimensionalidade

In [ ]:
df9.head()

In [ ]:
reducer = umap.UMAP(n_neighbors=90, n_jobs=-1)
embedding = reducer.fit_transform(df9)

df_viz['embedding_x'] = embedding[:, 0]
df_viz['embedding_y'] = embedding[:, 1]

In [ ]:
sns.scatterplot(x='embedding_x',
                y='embedding_y',
                hue='cluster',
                palette=sns.color_palette('hls', n_colors=len(df_viz['cluster'].unique())),
                data=df_viz)

## 9.2. Cluster profile

In [ ]:
df9['gross_revenue'].sum()

In [ ]:
df9[['cluster', 'gross_revenue']].groupby('cluster').sum() / df9['gross_revenue'].sum()

In [ ]:
# Number of customers
df_clusters = df9[['customer_id', 'cluster']].groupby('cluster').count().reset_index()

# Number of customers (%)
df_clusters['% customer_id'] = (df_clusters[['customer_id']] / df_clusters[['customer_id']].sum()) * 100

# Average gross revenue
aux1 = df9[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()

# % Gross revenue
aux2 = (df9[['cluster', 'gross_revenue']].groupby('cluster').sum() / df9['gross_revenue'].sum()).reset_index().rename(columns={'gross_revenue': '% gross_revenue'})

# Average recency
aux3 = df9[['recency', 'cluster']].groupby('cluster').mean().reset_index()

# Average invoice no
aux4 = df9[['invoice_no', 'cluster']].groupby('cluster').mean().reset_index()

# Average ticket
aux5 = df9[['avg_ticket', 'cluster']].groupby('cluster').mean().reset_index()

In [ ]:
df_clusters = pd.merge(df_clusters, aux1, on='cluster', how='left')
df_clusters = pd.merge(df_clusters, aux2, on='cluster', how='left')
df_clusters = pd.merge(df_clusters, aux3, on='cluster', how='left')
df_clusters = pd.merge(df_clusters, aux4, on='cluster', how='left')
df_clusters = pd.merge(df_clusters, aux5, on='cluster', how='left')
df_clusters

---

**Cluster 1 (candidatos à insiders)**:
- Número de customers: 267 (6,1%)
- Recência média: 20,28
- Média de compras: 19,5
- Média de receita: US$ 8.089,00
- Ticket médio: US$ 62,78
---

**Cluster 2**:
- Número de customers: 6 (0,14%)
- Recência média: 7,16
- Média de compras: 89
- Média de receita: US$ 182.181,98
- Ticket médio: US$ 253,62
---

**Cluster 3**:
- Número de customers: 28 (0,64%)
- Recência média: 6,18
- Média de compras: 57,82
- Média de receita: US$ 42.614,39
- Ticket médio: US$ 162,86
---

**Cluster 4**:
- Número de customers: 4071 (93,1%)
- Recência média: 96,96
- Média de compras: 3,64
- Média de receita: US$ 946,69
- Ticket médio: US$ 25,35
---

# 10. Deploy to Production